In [15]:
import numpy as np
from scipy.optimize import linprog

In [16]:
class Task:
    def __init__(self, A, b, c, lb, ub):
        self.A = A
        self.b = b
        self.c = c
        self.lb = lb
        self.ub = ub

    @staticmethod
    def compose_from_task(task):
        return Task(task.A, task.b, task.c, task.lb, task.ub)

In [17]:
class BranchAndBound:
    def __init__(self):
        self.solution = None
        self.f_value = -np.inf
        self.stack = []

    @staticmethod
    def solve_task(task: Task):
        return linprog(
            method='simplex',
            c=-task.c,
            b_ub=task.b,
            A_ub=task.A,
            bounds=list(zip(task.lb, task.ub))
        )

    @staticmethod
    def is_integer(x, eps=10 ** (-10)):
        return abs(x - round(x)) <= eps

    @staticmethod
    def split_task(task: Task, index: int, item: float):
        modified_lb = np.copy(task.lb)
        modified_ub = np.copy(task.ub)
        
        sign = 1 if(item >= 0) else -1
        modified_ub[index] = np.modf(item)[1]
        modified_lb[index] = int(item) + sign

        left_task = Task.compose_from_task(task)
        left_task.ub = modified_ub

        right_task = Task.compose_from_task(task)
        right_task.lb = modified_lb

        return left_task, right_task

    def solve(self, task):
        self.stack.append(task)

        while len(self.stack) != 0:
            current_task = self.stack.pop()

            simplex_res = self.solve_task(current_task)

            if simplex_res.status == 0:
                real_solution = simplex_res.x
                current_sum = -simplex_res.fun
                if all(self.is_integer(val) for val in real_solution):
                    if current_sum > self.f_value:
                        self.solution = real_solution
                        self.f_value = current_sum
                else:
                    for index, item in enumerate(real_solution):
                        if not self.is_integer(item) and current_sum > self.f_value:
                            left_task, right_task = self.split_task(current_task, index, item)
                            self.stack.append(left_task)
                            self.stack.append(right_task)

        return self.solution, self.f_value


In [18]:
A = np.array([
    [4, 3],
    [-4, 3]
])
b = np.array([22, 2])
c = np.array([-5, 4])
lb = np.array([1, 0])
ub = np.array([4, 5])

main_task = Task(A, b, c, lb, ub)

In [19]:
branch_and_bound_solver = BranchAndBound()
sol, res = branch_and_bound_solver.solve(main_task)

formatted_sol = ', '.join(map(str, map(int, sol)))
print('Result:')
print('Solution x: ', end='')
print(f'({formatted_sol})')
print(f'Max target function value: {res}')

Result:
Solution x: (1, 2)
Max target function value: 3.0
